In [1]:
%run "0_workspace_setup.ipynb"

In [2]:
from nhm_helpers.nhm_hydrofabric import make_hf_map_elements
from nhm_helpers.map_template import make_par_map
from nhm_helpers.nhm_assist_utilities import (
    make_plots_par_vals,
    create_append_gages_to_param_file,
    make_myparam_addl_gages_param_file,
)
from nhm_helpers.nhm_helpers import *
from ipywidgets import widgets
from IPython.display import display

# Import Notebook Packages
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, LineString
import warnings
from collections.abc import KeysView
import networkx as nx
from pyPRMS import ParameterFile
from pyPRMS.metadata.metadata import MetaData
from rich import pretty

pretty.install()
warnings.filterwarnings("ignore")

In [3]:
(
    hru_gdf,
    hru_txt,
    hru_cal_level_txt,
    seg_gdf,
    seg_txt,
    nwis_gages_aoi,
    poi_df,
    gages_df,
    gages_txt,
    gages_txt_nb2,
    HW_basins_gdf,
    HW_basins,
) = make_hf_map_elements(
    NHM_dir,
    model_dir,
    GIS_format,
    param_filename,
    control_file_name,
    nwis_gages_file,
    gages_file,
    default_gages_file,
    nhru_params,
    nhru_nmonths_params,
    nwis_gage_nobs_min,
)
con.print(
    f"{workspace_txt}\n",
    f"\n{gages_txt}{seg_txt}{hru_txt}",
    f"\n     {hru_cal_level_txt}\n",
    f"\n{gages_txt_nb2}",
)

NHM model domain: willamette_river, parameter file: myparam.param
Simulation and observation data range: 01/01/1979 - 12/31/2022 (from control.default.bandit).
 
The parameter file contains 100 gages managed by USGS, 329 segments, and 625 HRUs. 
     564 HRUs are within HWs, and 408 are within HW calibrated with streamflow observations.
 
 All gages have required metadata in the default_gages.csv.

#### The cell below makes a .csv file that is used to select additional gages to append the paramter file.

In [4]:
# def create_append_gages_to_param_file(
#     gages_df,
#     seg_gdf,
#     poi_df,
# ):
#     """
#     Make an editable .csv file from the gages_df, so that users can append new poigage (dimensioned) parameters to
#     the myparam.param file, and returns a pandas DataFrame of the written .csv.

#     First, a geopandas GeoDataFrame is made for the gages_df using the lat/lon from the gages_df (NWIS or user supplied).
#     Projection is set to crs=4326 and may introduce some spatial innaccuracy for older gages.
#     """
#     gages_gdf = gpd.GeoDataFrame(
#         gages_df,
#         geometry=gpd.points_from_xy(gages_df.longitude, gages_df.latitude),
#         crs=4326,
#     )
#     """ Gages_gdf (points_gdf) and seg_gdf (lines_gdf) projections changed for geo distance calculation. """
#     _points_gdf = gages_gdf.to_crs("ESRI:102039")
#     _lines_gdf = seg_gdf.to_crs("ESRI:102039")

#     _poi_max_distance = 1000  # spatial units of projections, meters for ESRI:102039

#     """ A spatial join for the nearest segment to a gage yields a likely candidate poi_gage_segment for each poi_gage_id
#         and the distance from the gage to the segment. """
#     append_gages_to_param_file_df = gpd.sjoin_nearest(
#         _points_gdf,
#         _lines_gdf,
#         max_distance=_poi_max_distance,
#         distance_col="distance",
#         how="left",
#     )
#     """ Cleanup """
#     append_gages_to_param_file_df = append_gages_to_param_file_df[
#         gages_df.columns.to_list() + ["nhm_seg", "model_idx", "distance"]
#     ]
#     append_gages_to_param_file_df = append_gages_to_param_file_df[
#         ["nhm_seg", "poi_name", "poi_agency", "distance"]
#     ].reset_index(drop=False)
#     append_gages_to_param_file_df.rename(
#         columns={"poi_id": "poi_gage_id"},
#         inplace=True,
#     )
#     """ Set an attribute "in_param_file" to show user which gages in the gages_df are in the myparam.param file. """
#     append_gages_to_param_file_df["in_param_file"] = "no"
#     param_file_gages = poi_df.poi_id.to_list()
#     append_gages_to_param_file_df.loc[
#         append_gages_to_param_file_df["poi_gage_id"].isin(param_file_gages),
#         "in_param_file",
#     ] = "yes"

#     """ Write new param file to the subdomain model directory. """
#     append_gages_to_param_file_df.to_csv(
#         model_dir / "append_gages_to_param_file.csv", index=False
#     )

In [5]:
create_append_gages_to_param_file(
    gages_df,
    seg_gdf,
    poi_df,
    model_dir,
)

In [6]:
## stop sigh, with alert :Open and edit CSV. Then run the cell below to add

In [7]:
# def make_myparam_addl_gages_param_file(
#     model_dir,
#     pdb,
# ):
#     """Read back in the modified gages to add file"""
#     col_names = [
#         "poi_gage_id",
#         "nhm_seg",
#     ]
#     col_types = [
#         np.str_,
#         "int32",
#     ]
#     cols = dict(zip(col_names, col_types))

#     addl_gages_df = pd.read_csv(
#         model_dir / "append_gages_to_param_file.csv",
#         dtype=cols,
#         usecols=[
#             "poi_gage_id",
#             "nhm_seg",
#         ],
#     )

#     nhm_seg_to_idx1 = {kk: vv + 1 for kk, vv in pdb.get("nhm_seg").index_map.items()}

#     addl_gages_df["poi_gage_segment"] = addl_gages_df["nhm_seg"].map(nhm_seg_to_idx1)

#     addl_gages = dict(
#         zip(
#             addl_gages_df["poi_gage_id"].to_list(),
#             addl_gages_df["poi_gage_segment"].to_list(),
#         )
#     )

#     pdb.add_poi(addl_gages)
#     pdb.write_parameter_file(model_dir / "myparam_addl_gages.param")
#     os.remove(model_dir / "append_gages_to_param_file.csv")
#     del pdb

#     return con.print(
#         "New paramter file `myparam_addl_gages.param` created in the model directory."
#     )

# Add POIs to parameters

In [8]:
make_myparam_addl_gages_param_file(
    model_dir,
    pdb,
)

Added user-specified POI streamgage (14147000) at segment index 23
Added user-specified POI streamgage (14158740) at segment index 122
Added user-specified POI streamgage (14158798) at segment index 124
Added user-specified POI streamgage (14164900) at segment index 100
Added user-specified POI streamgage (14171600) at segment index 68
Added user-specified POI streamgage (14182400) at segment index 162
Added user-specified POI streamgage (14185700) at segment index 179
Added user-specified POI streamgage (14187100) at segment index 172
Added user-specified POI streamgage (14187600) at segment index 169
Added user-specified POI streamgage (14196000) at segment index 220
Added user-specified POI streamgage (14200100) at segment index 245
Added user-specified POI streamgage (14200300) at segment index 246
Added user-specified POI streamgage (14200400) at segment index 249
Added user-specified POI streamgage (14200700) at segment index 248
Added user-specified POI streamgage (14202500) at 

The new parameter file myparam_addl_gages.param already exists and will NOT be overwritten. Please rename that file
and rerun this cell.

# To view thhe model with the new parameter file, update the `param_file` in [0_workspace_setup](./0_workspace_setup.ipynb). We strongly recommend renaming the new parameter file, delete the `notebook_output_files` folder in the model directory and delete the `output` folder in the model directory. Then, rerun all notebooks.